In [0]:

####    使用knn，预测用户的签到位置。
####    数据地址：https://www.kaggle.com/c/facebook-v-predicting-check-ins/data
####    时间复杂度o(n*k)：n为样本数量，k为单个样本特征的维度。如果不考虑特征维度的粒度为o(n)
####    空间复杂度o(n*k)：n为样本数量，k为单个样本特征的维度。如果不考虑特征维度的粒度为o(n)

# 指定当前的工作文件夹
import os

# 此处为google drive中的文件路径,drive为之前指定的工作根目录，要加上
os.chdir("gdrive/My Drive")

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd


def knncls():
  

  #读取数据
  data = pd.read_csv("./data/FBlocation/train.csv")
  print(data.head(10))
  
  #处理数据(因为预测的位置有很多，需要处理目标值)
  
  #1.缩小数据,查询数据刷选
  data = data.query("x > 1.0 & x < 1.25 & y > 2.5 & y < 2.7")
 
  #2.处理时间的数据，转为y-m-d h-m-s的格式
  time_value = pd.to_datetime(data['time'],unit='s')
  print(time_value)
  
  #3.把日期个数转换为字典格式
  time_value = pd.DatetimeIndex(time_value)
  
  #4.构造一些特征
  data['day'] = time_value.day
  data['hour'] = time_value.hour
  data['weekday'] = time_value.weekday
  
  #把时间戳特征(按列，sk-learn中0表示列)删除
  data.drop(['time'],axis=1)
  
  print(data)
  
  #把签到数量小于n个的目标位置删除
  place_count = data.groupby('place_id').count()
  tf = place_count[place_count.row_id > 3].reset_index()
  
  data = data[data['place_id'].isin(tf.place_id)]
  
  #取出数据当中的特征值和目标值
  y = data['place_id']
  x = data.drop(['place_id'],axis=1)
  x = data.drop(['row_id'],axis=1) 
  
  #进行数据的分割，划分为训练集和测试集
  x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.25)
  
  #特征工程（标准化）经过处理后的数据均值为0，标准差为1:  标准话和drop row_id之前准确率只有4%，之后准确率提高到了0.659369527145359
  std = StandardScaler()
  #对训练集和测试集的特征进行标准化,fit就是进行一次平均值和标准差
  x_train = std.fit_transform(x_train) 
  x_test = std.transform(x_test)  # 前面一步已经计算好了标准差和均值
  
  
  # 进行算法流程
  knn = KNeighborsClassifier(n_neighbors = 5)
  
  knn.fit(x_train,y_train)
  
  y_predict = knn.predict(x_test)
  print("预测目标的签到位置为;", y_predict)
  
  #得出准确率
  print("预测的准确率：", knn.score(x_test,y_test))
  
  return None
  
if __name__ == "__main__":
  knncls()


   row_id       x       y  accuracy    time    place_id
0       0  0.7941  9.0809        54  470702  8523065625
1       1  5.9567  4.7968        13  186555  1757726713
2       2  8.3078  7.0407        74  322648  1137537235
3       3  7.3665  2.5165        65  704587  6567393236
4       4  4.0961  1.1307        31  472130  7440663949
5       5  3.8099  1.9586        75  178065  6289802927
6       6  6.3336  4.3720        13  666829  9931249544
7       7  5.7409  6.7697        85  369002  5662813655
8       8  4.3114  6.9410         3  166384  8471780938
9       9  6.3414  0.0758        65  400060  1253803156
957        1970-01-10 02:11:10
4345       1970-01-05 15:08:02
4735       1970-01-06 23:03:03
6090       1970-01-02 16:25:07
6234       1970-01-04 15:52:57
7468       1970-01-09 15:26:06
8478       1970-01-08 23:52:02
12125      1970-01-07 03:55:07
20660      1970-01-08 03:08:15
20930      1970-01-02 21:31:48
26584      1970-01-04 15:48:09
27937      1970-01-08 03:51:54
30798      1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

            row_id       x       y  accuracy  ...    place_id  day  hour  weekday
957            957  1.1832  2.6891        58  ...  6683426742   10     2        5
4345          4345  1.1935  2.6550        11  ...  6889790653    5    15        0
4735          4735  1.1452  2.6074        49  ...  6822359752    6    23        1
6090          6090  1.1140  2.6262        11  ...  4000153867    2    16        4
6234          6234  1.1449  2.5003        34  ...  3741484405    4    15        6
7468          7468  1.0058  2.5096        66  ...  9076695703    9    15        4
8478          8478  1.2015  2.5187        72  ...  3992589015    8    23        3
12125        12125  1.1388  2.5029        69  ...  7536975002    7     3        2
20660        20660  1.2387  2.5959        65  ...  3683087833    8     3        3
20930        20930  1.0519  2.5208        67  ...  6399991653    2    21        4
26584        26584  1.1235  2.6282        63  ...  5606572086    4    15        6
27937        279